# Фабула

In [25]:
class FileWrapper:
    def __init__(self, path, mode='rt'):
        self._file = open(path, mode)
        self._read_cnt = 0
    
    def _on_read(self):
        self._read_cnt += 1
        if self._read_cnt >= 2:
            raise ValueError("too much")
    
    def read(self, size=-1, /):   
        self._on_read()
        return self._file.read(size)
    
    def readline(self):
        self._on_read()
        return self._file.readline()

    def close(self):
        self._file.close()
    
    def __iter__(self):
        line = self.readline()
        while line:
            yield line
            line = self.readline()

In [11]:
#here-document

In [12]:
%%bash
cat <<EOF > ./file.txt
first line
second line
third line
EOF

In [13]:
# !echo "first line
# second line
# third line" | cat > ./file.txt

In [14]:
!cat ./file.txt

first line
second line
third line


In [15]:
for line in open('./file.txt'):
    print(line)

first line

second line

third line



In [17]:
f = FileWrapper('./file.txt')
for line in f:
    print('hey, line is', line)

hey, line is first line



ValueError: too much

In [19]:
f._file

<_io.TextIOWrapper name='./file.txt' mode='rt' encoding='UTF-8'>

In [ ]:
f = open('./file.txt')

## Что делать? Обложиться try / except!

In [22]:

try:
    raise ValueError('hello')
except ValueError as e:
    print('value error')
except Exception as e:
    print(e)
finally:
    print('finally')


value error
finally


In [26]:
f = FileWrapper('./file.txt')
try:
    for line in f:
        pass
finally:
    f.close()
    

ValueError: too much

# Хотим "закрывать ресурсы"

## Как это делают в других языках?

### Golang: defer

```go
package main

import "fmt"

func main() {
    defer fmt.Println("world")
    fmt.Println("hello")
}
```


### C++: RAII (Resource Acquisition Is Initialization, [link](https://en.cppreference.com/w/cpp/language/raii)

```c++
class RAIIResource {
    TResource* _p_resource;

    RAIIResource(...) {
        _p_resource = new TResource(...blablabla...);
    }

    ~RAIIResource() {
        delete _p_resource;
    }
};

void usage()
{
    RAIIResource resource(...blabla...);
    throw std::runtime_error("ahhahah");
    // On stack unwinding, resource object is deallocated -> destructor called -> resource freed
}

//
// another example (cppreference.com)
//

std::mutex m;
 
void bad() 
{
    m.lock();                    // acquire the mutex
    f();                         // if f() throws an exception, the mutex is never released
    if(!everything_ok()) return; // early return, the mutex is never released
    m.unlock();                  // if bad() reaches this statement, the mutex is released
}
 
void good()
{
    std::lock_guard<std::mutex> lk(m); // RAII class: mutex acquisition is initialization
    f();                               // if f() throws an exception, the mutex is released
    if(!everything_ok()) return;       // early return, the mutex is released
}                                      //                            // if good() returns normally, the mutex is released

```

### Java: [try-with-resources](https://docs.oracle.com/javase/tutorial/essential/exceptions/tryResourceClose.html)

```java
static String readFirstLineFromFile(String path) throws IOException {
    try (BufferedReader br =
                   new BufferedReader(new FileReader(path))) {
        return br.readLine();
    }
}
```

# Чем плохо try-except?

In [ ]:
def some_generator():
    resource = SomeResource()
    try:
        yield from source1
        
        resource2 = SomeResource2():
        try:
            yield from list('abcdef')
        finally:
            resource2.close()
    
    finally:
        resource.close()

- Ресурсом владеет SomeResource, но закрывает его пользователь!
- "Лапша из кода", помноженное на "часто надо"

In [ ]:
def some_generator():
    with SomeResource() as resource:
        yield from source1
        
        with SomeResource2() as resource2:
            yield from list('abcdef')

In [ ]:
with obj as value:
    code body


value = obj.__enter__()

code body

value.__exit__()


# Scoping Warn

In [27]:
def foo():
    a = 5

print(a) # NameError: name 'a' is not defined

NameError: name 'a' is not defined

In [28]:
with some_context_manager as value:
    a = 5

print(a)  # works

NameError: name 'some_context_manager' is not defined

# Context Managers

In [39]:
class MyContextManager:
    def __enter__(self):
        print('__enter__')
        return 'privet'
    def __exit__(self, exc_type, exc_value, traceback):
        print('__exit__')
        print(locals())
        return True

In [40]:
#bool(cm.__exit__(...))

In [41]:
with MyContextManager() as value:
    print('code body, value is ', value)
    raise ValueError('some exception')
    print('never should be here')

__enter__
code body, value is  privet
__exit__
{'self': <__main__.MyContextManager object at 0x7fbd11803130>, 'exc_type': <class 'ValueError'>, 'exc_value': ValueError('some exception'), 'traceback': <traceback object at 0x7fbd119ea200>}


# Типичные сценарии использования

## файлы

In [ ]:
data = open('./file.txt').read()   # WRONG

with open('./file.txt') as f:    # APPROVED
    data = f.read()

## инструментирование

In [ ]:
with metric.SUBROUTINE_LATENCY_HISTOGRAM.measure():
    # do some long stuff

# metric.SUBROUTINE_LATENCY updated

## любые операции, которые 100% должны быть "отменены"

In [ ]:
with redirect_stdout_to(open('./file.txt')):
    print('ahhhaha')  # no stdout, but print into './file.txt'

# Множественные ресурсы

In [ ]:
with cm1[ as value1], cm2[ as value2], cm3[ as value3]:
    pass  # do stuff

## Что делать, если ресурсов слишком много или они опциональные?

Обсудим на семинаре!

# Typing

In [48]:
from typing import ContextManager, Iterator
from contextlib import AbstractContextManager  # посмотреть код на семинаре!

class MyCM(ContextManager[str]):
    def __enter__(self) -> str:
        return 'ahhahah'

    def __exit__(self, *exc_args) -> None:
        pass
        
    
cm: ContextManager[str]
with cm as value:
    # value: str
    pass

NameError: name 'cm' is not defined

In [ ]:
isinstance(MyCM(), ContextManager)

In [47]:
isinstance(MyCM(), AbstractContextManager)

False

# Быстрый способ сделать контекстный менеджер

In [59]:
from contextlib import contextmanager

@contextmanager
def generator():
    print('setup')
    try:
        yield 'hello'
    finally:
        print('teardown')
        
        
## Что делать, если я хочу умулировать __exit__ : return True ??

@contextmanager
def generator():
    print('setup')
    try:
        yield 'hello'
    except Exception as e:
        pass
    finally:
        print('teardown')

with generator() as value:
    print('code body, value is', value)
    raise ValueError('some exception')

setup
code body, value is hello
teardown


In [ ]:
# @deco
# def foo():
#     ...

# # equal to

# def foo():
#     ...

# foo = deco(foo)

# Задачи на семинар

In [ ]:
def example():
    # обоими способами
    timeit

In [ ]:
# обоими способами

def example():
    some_dict = dict(a=1, b=2)
    
    with popped_key(some_dict, 'a'):
        print(some_dict)  # dict(b=2)
    
    with popped_key(some_dict, 'b'):
        print(some_dict)  # dict(a=1)

In [1]:
import sys

In [ ]:
sys.__st